# Big Data Platform
## Assignment 2: MapReduce

**By:**  

Oren Ben Eliyahu, 204079453
<br>Yuval Barkan, 205714447

<br><br>

**The goal of this assignment is to:**
- Understand and practice the details of MapReduceEngine

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading a Jupyter notebook.
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.
  
  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q1 - 5 points - Initial Steps
- Q2 - 50 points - MapReduceEngine
- Q3 - 30 points - Implement the MapReduce Inverted index of the JSON documents
- Q4 - 5 points - Testing Your MapReduce
- Q5 - 10 points - Final Thoughts 

`Total: 100`

**Prerequisites**

In [1]:
# example
!pip install --quiet zipfile36

**Imports**

In [2]:
# general
import os
import time
import random
import warnings
import threading # you can use easier threading packages
import sqlite3
from sqlite3 import Error

# ml
import numpy as np
import scipy as sp
import pandas as pd

# visual
import seaborn as sns
import matplotlib.pyplot as plt

# notebook
from IPython.display import display

**Hide Warnings**

In [3]:
warnings.filterwarnings('ignore')

**Disable Autoscrolling**

In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

**Set Random Seed**

In [5]:
random.seed(123)


# Question 1
# Initial Steps



Use python to Create `mapreducetemp` and `mapreducefinal` folders

<b><u>Answer - </u></b>
<br>We first created the folder for the generated csv files in addition to mapreducetemp and mapreducefinal folders. 

In [6]:
def create_folder(parent_dir,folder_names):
    for i in folder_names:
        path = os.path.join(parent_dir, i)
        os.mkdir(path)
        print(f"{i} folder created")

In [7]:
parent_dir = os.getcwd()
folder_names = ['csv_files','mapreducetemp','mapreducefinal']

create_folder(parent_dir,folder_names)

csv_files folder created
mapreducetemp folder created
mapreducefinal folder created


Write Python code to create 20 different CSV files in this format:  `myCSV[Number].csv`, where each file contains 10 records. 

The schema is `(‘firstname’,’secondname’,city’)`  

Values should be randomly chosen from the lists: 
- `firstname` : `[John, Dana, Scott, Marc, Steven, Michael, Albert, Johanna]`  
- `city` : `[New York, Haifa, München, London, Palo Alto,  Tel Aviv, Kiel, Hamburg]`  
- `secondname`: any value  

In [8]:
firstname = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
city = ['NewYork', 'Haifa', 'Munchen', 'London', 'PaloAlto',  'TelAviv', 'Kiel', 'Hamburg']
secondname = None 

In [9]:
num_of_rows = 10
num_of_files = 20

cols = ['firstname','secondname','city']
firstname = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
city = ['New York', 'Haifa', 'München', 'London', 'Palo Alto',  'Tel Aviv', 'Kiel', 'Hamburg'] 
      
def create_csv(num_of_csv_files:int, cols_name, num_of_rows:int):
    for i in range(num_of_csv_files):
        data_dict = {
            cols_name[0]: np.random.choice(firstname, num_of_rows),
            cols_name[1]: np.random.choice(firstname, num_of_rows),
            cols_name[2]: np.random.choice(city, num_of_rows)
        }
        df = pd.DataFrame(data_dict)
        df.to_csv(f'csv_files/myCSV{i+1}.csv', index=False)
        
    print(f"All {num_of_csv_files} CSV files were created")
    
create_csv(num_of_files,cols,num_of_rows)

All 20 CSV files were created


<br><br><br>
# Question 2
## MapReduceEngine

Write Python code to create an SQLite database with the following table

`TableName: temp_results`   
`schema: (key:TEXT,value:TEXT)`

In [10]:
def create_connection(db_data):
    try:
        conn = sqlite3.connect(db_data)
        print("Establish connection")
    except Error as e:
        print(e)
    finally:
        return conn


def create_table(conn, create_table_query):
    try:
        c = conn.cursor()
        c.execute(create_table_query)
    except Error as e:
        print(e)
        
def query(conn, select_query):
    try:
        c = conn.cursor()
        select_all = select_query
        rows = c.execute(select_all).fetchall()
        # Output to the console screen
        return rows
    
    except Error as e:
        print(e)

In [11]:
MYDATA_DB = 'temp_results.db'

create_table_temp_results = '''CREATE TABLE IF NOT EXISTS temp_results(
                                key TEXT,
                                value TEXT)
                            '''

conn = create_connection(MYDATA_DB)
create_table(conn,create_table_temp_results)

Establish connection


1. **Create a Python class** `MapReduceEngine` with method `def execute(input_data, map_function, reduce_function)`, such that:
    - `input_data`: is an array of elements
    - `map_function`: is a pointer to the Python function that returns a list where each entry of the form (key,value) 
    - `reduce_function`: is pointer to the Python function that returns a list where each entry of the form (key,value)

<br><br>

**Implement** the following functionality in the `execute(...)` function:

<br>

1. For each key  from the  input_data, start a new Python thread that executes map_function(key) 
<br><br>
2. Each thread will store results of the map_function into mapreducetemp/part-tmp-X.csv where X is a unique number per each thread.
<br><br>
3. Keep the list of all threads and check whether they are completed.
<br><br>
4. Once all threads completed, load content of all CSV files into the temp_results table in SQLite.

    Remark: Easiest way to loop over all CSV files and load them into Pandas first, then load into SQLite  
    `data = pd.read_csv(path to csv)`  
    `data.to_sql(‘temp_results’,sql_conn, if_exists=’append’,index=False)`
<br><br>

5. **Write SQL statement** that generates a sorted list by key of the form `(key, value)` where value is concatenation of ALL values in the value column that match specific key. For example, if table has records
<table>
    <tbody>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV1.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">Dana</td>
                <td style="text-align:center">myCSV5.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV7.csv</td>
            </tr>
    </tbody>
</table>

    Then SQL statement will return `(‘John’,’myCSV1.csv, myCSV7.csv’)`  
    Remark: use GROUP_CONCAT and also GROUP BY ORDER BY
<br><br><br>
6. **Start a new thread** for each value from the generated list in the previous step, to execute `reduce_function(key,value)` 
<br>    
7. Each thread will store results of reduce_function into `mapreducefinal/part-X-final.csv` file  
<br>
8. Keep list of all threads and check whether they are completed  
<br>
9. Once all threads completed, print on the screen `MapReduce Completed` otherwise print `MapReduce Failed` 



In [12]:
import concurrent.futures as cf

class MapReduceEngine():
    
    def execute(self, input_data, map_function, reduce_function, params):
                        
        # map function
        threads = []
        
        for count, csv_file in enumerate(input_data):
            with cf.ThreadPoolExecutor() as executor:
                t = executor.submit(map_function, csv_file, params)             
                df = t.result()
                df.to_csv(f'mapreducetemp/part-tmp-{count+1}.csv', index=False)
                threads.append(t)
                
        # Check if all threads are completed:   
        if len(threads) != len(input_data):
            print("MapReduce Failed")
            return

            
        threads.clear()

        # Load contect of all CSV files into the temp_results table
        for i in range(len(input_data)):
            data = pd.read_csv(f'mapreducetemp/part-tmp-{i+1}.csv')
            data.to_sql('temp_results',conn, if_exists='append',index=False)
        
        # SQL statements:
        grouping_query = "SELECT key, GROUP_CONCAT(value,',') FROM temp_results GROUP BY key"
        reduce_input_values = query(conn,grouping_query)
        
        unique_keys = "SELECT  count(distinct key) FROM temp_results"
        reduce_threads = query(conn,unique_keys)[0][0]
        
        # reduce function
        threads = []
        
        for i in range(reduce_threads):
            with cf.ThreadPoolExecutor() as executor:
                key = reduce_input_values[i][0]
                values = reduce_input_values[i][1].split(',')
                
                t = executor.submit(reduce_function, key, values)             
                df = t.result()
                df.to_csv(f'mapreducefinal/part-{i+1}-final.csv', index=False)
                threads.append(t)
        
        # Check if all threads are completed:   
        if len(threads) != reduce_threads:
            print("MapReduce Failed")
            return
        
        else:
            print("MapReduce Completed")         

<br><br><br><br>

# Question 3
## Implement the MapReduce Inverted index of the JSON documents

Implement a function `inverted_map(document_name)` which reads the CSV document from the local disc and return a list that contains entries of the form (key_value, document name).

For example, if myCSV4.csv document has values like:  
`{‘firstname’:’John’,‘secondname’:’Rambo’,‘city’:’Palo Alto’}`

Then `inverted_map(‘myCSV4.csv’)` function will return a list:  
`[(‘firstname_John’,’ myCSV4.csv’),(‘secondname_Rambo’,’ myCSV4.csv’), (‘city_Palo Alto’,’ myCSV4.csv’)]`

In [13]:
# The goal of map function is to create csv files. 
# Therfore we chose to optimize the process and not create unnecessary variables (such as list) and return a data frame. 

def inverted_map(document_name:str, column_index:dict):
    csv_dir = f'csv_files/{document_name}'
    file = pd.read_csv(csv_dir)
    df = pd.DataFrame({'key':file.iloc[:,column_index['column']],'value':csv_dir})
    return df 


Write a reduce function `inverted_reduce(value, documents)`, where the field “documents” contains a list of all CSV documents per given value.   
This list might have duplicates.   
Reduce function will return new list without duplicates.

For example,  
calling the function `inverted_reduce(‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’)`   
will return a list `[‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’]`

In [14]:
# The goal of reduce function is to create csv files. 
# Therfore we chose to optimize the process and not create unnecessary variables (such as list) and return a data frame. 

def inverted_reduce(key,documents):
    df = pd.DataFrame({'key':key,'value':list(set(documents))})
    return df



<br><br><br><br>
# Question 4
## Testing Your MapReduce

**Create Python list** `input_data` : `[‘myCSV1.csv’,.. ,‘myCSV20.csv’]`

In [15]:
from os import listdir

def find_csv_filenames(path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [16]:
input_data = find_csv_filenames('csv_files/')

**Submit MapReduce as follows:**

In [17]:
mapreduce = MapReduceEngine()
params = {'column':2}
status = mapreduce.execute(input_data, inverted_map, inverted_reduce, params)

MapReduce Completed


Make sure that `MapReduce Completed` should be printed and `mapreducefinal` folder should contain the result files.

**Use python to delete all temporary data from mapreducetemp folder and delete SQLite database:**

In [18]:
# Droping the table:
query(conn,'''DROP TABLE temp_results''')

# Delete all temporary data from mapreducetemp folder
import glob

parent_dir = os.getcwd()
folder_dir = str(parent_dir)+'/mapreducetemp/*'

files = glob.glob(folder_dir)
for f in files: 
    os.remove(f)

<br><br><br><br>

# Question 5
# Final Thoughts

The phase where `MapReduceEngine` reads all temporary files generated by maps and sort them to provide each reducer a specific key is called the **shuffle step**.

Please explain **clearly** what would be the main problem of MapReduce when processing Big Data, if there is no shuffle step at all, meaning reducers will directly read responses from the mappers.

<u><b>Answer</b></u>
<br>The main goal in MapReduce is to perform calculations on distributed computers and get a result without duplications.
<br>The main problem in MapReduce without the shuffle step is that the same keys will likely to be sent to different reducers. This results duplication in the part that we want to elimante from it. 
<br>The shuffle step guarantees that each reducer will get all values from the same key, in this way the data can be reduced properly. 

<br><br><br><br>
Good Luck :)